# Inspect Trained Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np
from PIL import Image
import numpy as np
import os
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.datasets.utils as dataset_utils

from gendis.datasets import CausalMNIST

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
seed = 1
graph_type = 'chain'

results_dir = Path(f'/Users/adam2392/pytorch_data/data/chain-seed={seed}/seed-{seed}_{graph_type}/')
model = torch.load(results_dir / f'model.pt', map_location=torch.device('cpu'))

# Demo Trained Model Reproducing Images

Here, we are interested in qualitatively analyzing the trained model. This will entail

In [4]:
# get the images
graph_type = 'chain'
root = '/Users/adam2392/pytorch_data/'

dataset = CausalMNIST(root=root, graph_type=graph_type, label=0, download=True, train=True, n_jobs=None, intervention_idx=None)
dataset.prepare_dataset(overwrite=False)

img, metadata = dataset[0]
print(img.shape)

Loading dataset from "/Users/adam2392/pytorch_data/CausalMNIST/chain/chain-0-train.pt"
torch.Size([3, 28, 28]) {'width': tensor([0.5344]), 'color': tensor([0.3977]), 'fracture_thickness': tensor([8.9378]), 'fracture_num_fractures': tensor([1.]), 'label': 0, 'intervention_targets': [0, 0, 0]}
torch.Size([3, 28, 28])


In [10]:
# print(model)
print(model.encoder)

NonparametricClusteredCausalEncoder(
  (q0): NonparametricClusteredCausalDistribution(
    (q0): MultiEnvBaseDistribution()
    (flows): ModuleList(
      (0-2): 3 x AutoregressiveRationalQuadraticSpline(
        (mprqat): MaskedPiecewiseRationalQuadraticAutoregressive(
          (autoregressive_net): MADE(
            (preprocessing): Identity()
            (initial_layer): MaskedLinear(in_features=2352, out_features=128, bias=True)
            (blocks): ModuleList(
              (0-2): 3 x MaskedResidualBlock(
                (linear_layers): ModuleList(
                  (0-1): 2 x MaskedLinear(in_features=128, out_features=128, bias=True)
                )
                (activation): ReLU()
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (final_layer): MaskedLinear(in_features=128, out_features=54096, bias=True)
          )
        )
      )
    )
  )
  (flows): ModuleList(
    (0-2): 3 x AutoregressiveRationalQuadraticSpline(
  